In [1]:
import nibabel as nib
import numpy as np
from scipy.ndimage import zoom
import matplotlib.pyplot as plt

In [2]:
brain_image1_path = "/NAS/dumbo/protocoles/HAMSI/OurData4Mispel_T1Gado/Brains/CHU/Pat8_T1Gado_CHU_brain.nii.gz"
brain_image2_path = "/NAS/dumbo/protocoles/HAMSI/OurData4Mispel_T1Gado/Brains_Registered/COL_registred_on_CHU/Pat8_COL_registered.nii.gz"

# Paths to the masks
mask1_path = "/NAS/dumbo/protocoles/HAMSI/OurData4Mispel_T1Gado/Before_prettt_mask/CHU/Pat8_T1Gado_CHU_bet_mask.nii.gz"
mask2_path = "/NAS/dumbo/protocoles/HAMSI/OurData4Mispel_T1Gado/Masks_Registered/COL_mask_registred/Pat8_COL_mask_registered.nii.gz"

# Load the images and masks
brain_image1 = nib.load(brain_image1_path)
brain_image2 = nib.load(brain_image2_path)
mask1 = nib.load(mask1_path).get_fdata()
mask2 = nib.load(mask2_path).get_fdata()

In [3]:
mean1 = np.mean(brain_image1.get_fdata()[mask1 > 0])
std1 = np.std(brain_image1.get_fdata()[mask1 > 0])
mean2 = np.mean(brain_image2.get_fdata()[mask2 > 0])
std2 = np.std(brain_image2.get_fdata()[mask2 > 0])

# Z-score normalization for both images
normalized_data1 = (brain_image1.get_fdata() - mean1) / std1
normalized_data2 = (brain_image2.get_fdata() - mean2) / std2

# Create new NIfTI images with the normalized data
normalized_image1 = nib.Nifti1Image(normalized_data1, affine=brain_image1.affine)
normalized_image2 = nib.Nifti1Image(normalized_data2, affine=brain_image2.affine)



In [ ]:
def bland_altman_plot(data1, data2):
    

    
    mean = (data1.get_fdata() + data2.get_fdata()) / 2
    diff = data1.get_fdata() - data2.get_fdata()

    mean_diff = np.mean(diff)
    std_diff = np.std(diff, ddof=1)
    lower_limit = mean_diff - 1.96 * std_diff
    upper_limit = mean_diff + 1.96 * std_diff

    
    non_zero_diff = diff[diff != 0]

    #percent_non_zero_diff = len(non_zero_diff) / len(diff) * 100

    plt.figure(figsize=(8, 6))
    plt.scatter(mean, diff, c='b', alpha=0.5, s=30)
    plt.axhline(mean_diff, color='red', linestyle='--', linewidth=2)
    plt.axhline(lower_limit, color='green', linestyle='--', linewidth=1)
    plt.axhline(upper_limit, color='green', linestyle='--', linewidth=1)
    plt.xlabel('Moyenne des deux mesures')
    plt.ylabel('Différence entre les mesures')
    plt.title('Bland-Altman Plot')
    plt.grid(True)

    #plt.text(0.05, 0.1, f'Non-zero differences: {percent_non_zero_diff:.2f}%', transform=plt.gca().transAxes)
    
    # Show the plot
    plt.show()

# Create the Bland-Altman plot for the normalized images
bland_altman_plot(normalized_image1, normalized_image2)